# Recall By Judin

## Data preprocessing

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("scrubbed.csv", low_memory=False)
data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


## Cleaning the data

In [4]:
data = data.rename({'duration (seconds)': 'duration_seconds', 'duration (hours/min)': 'duration_hours_min', 'date posted': 'date_posted'}, axis=1)

In [5]:
data.dtypes

datetime               object
city                   object
state                  object
country                object
shape                  object
duration_seconds       object
duration_hours_min     object
comments               object
date_posted            object
latitude               object
longitude             float64
dtype: object

In [6]:
data.shape

(80332, 11)

In [7]:
data.datetime = data.datetime.str.replace('24:', '00:')

In [8]:
data.datetime = data.datetime.astype('datetime64')

In [9]:
duration_without_na = data.duration_seconds.dropna()
duration_without_na[~duration_without_na.str.contains('^\d+(\.\d+)?$', na=False)].value_counts()

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_5828/1235157003.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  duration_without_na[~duration_without_na.str.contains('^\d+(\.\d+)?$', na=False)].value_counts()


2`           1
8`           1
2631600      1
0.5`         1
Name: duration_seconds, dtype: int64

In [10]:
data.duration_seconds = data.duration_seconds.str.replace('`', '').astype('float64')

In [11]:
data.date_posted = data.date_posted.astype('datetime64')

In [12]:
data.date_posted

0       2004-04-27
1       2005-12-16
2       2008-01-21
3       2004-01-17
4       2004-01-22
           ...    
80327   2013-09-30
80328   2013-09-30
80329   2013-09-30
80330   2013-09-30
80331   2013-09-30
Name: date_posted, Length: 80332, dtype: datetime64[ns]

In [13]:
data.latitude[~data.latitude.str.contains('^-?\d+(\.\d+)?$')]

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_5828/966619224.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data.latitude[~data.latitude.str.contains('^-?\d+(\.\d+)?$')]


43782    33q.200088
Name: latitude, dtype: object

In [14]:
data = data.drop(index=43782)

In [15]:
data.latitude = data.latitude.astype('float64')

In [16]:
data.dtypes

datetime              datetime64[ns]
city                          object
state                         object
country                       object
shape                         object
duration_seconds             float64
duration_hours_min            object
comments                      object
date_posted           datetime64[ns]
latitude                     float64
longitude                    float64
dtype: object

In [17]:
data.isna().any()

datetime              False
city                  False
state                  True
country                True
shape                  True
duration_seconds      False
duration_hours_min    False
comments               True
date_posted           False
latitude              False
longitude             False
dtype: bool

## Start here....

Visualize the first 6 rows of the dataset.

In [18]:
# add your code below to get the first 6 rows of the dataset
data.head(6)

,datetime,city,state,country,shape,duration_seconds,duration_hours_min,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611
5,1961-10-10 19:00:00,bristol,tn,us,sphere,300.0,5 minutes,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.595000,-82.188889


In [19]:
#Get some general information about the dataset.
data.info

<bound method DataFrame.info of                  datetime                  city state country     shape  \
0     1949-10-10 20:30:00            san marcos    tx      us  cylinder   
1     1949-10-10 21:00:00          lackland afb    tx     NaN     light   
2     1955-10-10 17:00:00  chester (uk/england)   NaN      gb    circle   
3     1956-10-10 21:00:00                  edna    tx      us    circle   
4     1960-10-10 20:00:00               kaneohe    hi      us     light   
...                   ...                   ...   ...     ...       ...   
80327 2013-09-09 21:15:00             nashville    tn      us     light   
80328 2013-09-09 22:00:00                 boise    id      us    circle   
80329 2013-09-09 22:00:00                  napa    ca      us     other   
80330 2013-09-09 22:20:00                vienna    va      us    circle   
80331 2013-09-09 23:00:00                edmond    ok      us     cigar   

       duration_seconds duration_hours_min  \
0                2700

It seems like the column duration_hours_min is not that useful and you would like to drop the column.

In [20]:
# add your code here in the line below
data.drop("duration_hours_min", axis=1, inplace=True)
# show the state of the current data
data.head()

,datetime,city,state,country,shape,duration_seconds,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611


In [21]:
data.head()

,datetime,city,state,country,shape,duration_seconds,comments,date_posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,1956-10-10 21:00:00,edna,tx,us,circle,20.0,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611


Get the summary for the data.

In [22]:
# identify the suitable method for the use case below
# data._________(datetime_is_numeric=True)
data.describe(datetime_is_numeric=True)

,datetime,duration_seconds,date_posted,latitude,longitude
count,80331,8.033100e+04,80331,80331.000000,80331.000000
mean,2004-05-17 07:11:29.266161024,9.016999e+03,2007-07-28 01:44:20.776038912,38.124477,-86.772650
min,1906-11-11 00:00:00,1.000000e-03,1998-03-07 00:00:00,-82.862752,-176.658056
25%,2001-08-02 22:20:00,3.000000e+01,2003-11-26 00:00:00,34.134722,-112.073333
50%,2006-11-22 05:10:00,1.800000e+02,2007-11-28 00:00:00,39.411111,-87.903611
75%,2011-06-21 01:00:00,6.000000e+02,2011-10-10 00:00:00,42.788333,-78.755000
max,2014-05-08 18:45:00,9.783600e+07,2014-05-08 00:00:00,72.700000,178.441900
std,NaN,6.202206e+05,NaN,10.469636,39.697396


How many sightings are there in this dataset?

In [23]:
data.shape

(80331, 10)

Which city has the most UFO sightings?

In [24]:
data.city.value_counts()

seattle                                  525
phoenix                                  454
portland                                 374
las vegas                                368
los angeles                              353
                                        ... 
playa del cura (grand canaria island)      1
livermore/pleasonton (area)                1
hamburg (germany)                          1
cole camp                                  1
hamstead (hollyridge)                      1
Name: city, Length: 19899, dtype: int64

Does Malaysia have UFO sightings?

In [25]:
# add your code in to list down all unique countries in the dataset
data.country.unique()

array(['us', nan, 'gb', 'ca', 'au', 'de'], dtype=object)

What is the average time in minutes of all the sightings?

In [26]:
data.duration_seconds.mean() / 60

150.2833170384617

Which state in the United States has the most number of UFO sightings?

In [27]:
data[data.country == 'us'].state.value_counts().head()

ca    8912
wa    3966
fl    3835
tx    3447
ny    2980
Name: state, dtype: int64

What is the min duration of the sightings in states in the United States?

In [28]:
data[data.country == 'us'].groupby('state').duration_seconds.min()

state
ak    1.00
al    0.50
ar    0.20
az    0.10
ca    0.10
co    0.10
ct    1.00
dc    4.00
de    1.00
fl    1.00
ga    0.10
hi    1.00
ia    0.50
id    0.30
il    0.01
in    0.50
ks    1.00
ky    1.00
la    1.00
ma    0.50
md    0.30
me    0.40
mi    0.50
mn    0.33
mo    0.50
ms    1.00
mt    1.00
nc    0.05
nd    1.00
ne    1.00
nh    0.50
nj    0.50
nm    0.50
nv    0.45
ny    0.05
oh    0.50
ok    0.50
or    0.01
pa    0.50
pr    2.00
ri    0.50
sc    0.50
sd    1.50
tn    1.00
tx    0.50
ut    0.50
va    1.00
vt    1.00
wa    0.01
wi    1.00
wv    1.00
wy    1.00
Name: duration_seconds, dtype: float64

What are all the UFO shapes?

In [29]:
data['shape'].unique()

array(['cylinder', 'light', 'circle', 'sphere', 'disk', 'fireball',
       'unknown', 'oval', 'other', 'cigar', 'rectangle', 'chevron',
       'triangle', 'formation', nan, 'delta', 'changing', 'egg',
       'diamond', 'flash', 'teardrop', 'cone', 'cross', 'pyramid',
       'round', 'crescent', 'flare', 'hexagon', 'dome', 'changed'],
      dtype=object)

What is the probability of seeing a circle UFO?

In [30]:
# get the number of circle UFO observed in the dataset and assign it to sum_of_circles
sum_of_circles = (data['shape'] == 'circle').sum()
# exclude na and get the total observations and assign to sum_of_not_na
sum_of_not_na = data['shape'].notnull().sum()
# calculate the probability with the sum_of_circles divided by sum_of_not_na
sum_of_circles / sum_of_not_na

0.09704205410783301

Do we have sightings with same comments?

In [32]:
# add your code here
(data.comments.value_counts() > 1).any()
# you should get only a boolean value indicating whether there are sightings with the same comments

True

Which year has the most UFO sightings?

In [33]:
# uncomment and complete the code below
data['year'] = data.datetime.dt.year
data['month'] = data.datetime.dt.month
data['day'] = data.datetime.dt.day

In [34]:
data.year.value_counts()

2012    7357
2013    7037
2011    5107
2008    4820
2009    4541
        ... 
1920       1
1916       1
1906       1
1925       1
1934       1
Name: year, Length: 87, dtype: int64

At what time in a day should we look out the window to search for UFO?

In [35]:
# complete the code below
data['hour'] = data.datetime.dt.hour

In [36]:
# add your code below to answer the question
data.hour.value_counts().sort_index()

0      4802
1      3210
2      2357
3      2004
4      1529
5      1590
6      1224
7       905
8       803
9       958
10     1166
11     1144
12     1368
13     1303
14     1322
15     1433
16     1620
17     2592
18     4002
19     6147
20     8617
21    11445
22    10837
23     7953
Name: hour, dtype: int64

What shape of UFO can we see more often in the morning (0600-1159)?

In [37]:
from datetime import time;
data['time'] = data.datetime.dt.time
morning_sightings = data[(data.time >= time(hour=6)) & (data.time < time(hour=12))]
morning_sightings.head()

,datetime,city,state,country,shape,duration_seconds,comments,date_posted,latitude,longitude,year,month,day,hour,time
35,1982-10-10 07:00:00,gisborne (new zealand),NaN,NaN,disk,120.0,gisborne nz 1982 wainui beach to sponge bay,2002-01-11,-38.662334,178.017649,1982,10,10,7,07:00:00
84,1999-10-10 06:00:00,dallas,tx,us,other,10.0,group of twinkling lights at high altitude,2001-11-20,32.783333,-96.800000,1999,10,10,6,06:00:00
85,1999-10-10 11:00:00,san diego,ca,us,fireball,3600.0,At 10:45 in the morning&#44 my father and I no...,1999-10-19,32.715278,-117.156389,1999,10,10,11,11:00:00
94,2000-10-10 06:15:00,boynton beach,fl,us,other,10.0,Unusual light formation moving extremely fast ...,2000-12-02,26.525000,-80.066667,2000,10,10,6,06:15:00
95,2000-10-10 07:30:00,victoria (canada),bc,ca,cylinder,30.0,Smooth Shiny Cylinder,2000-12-02,46.216667,-63.483333,2000,10,10,7,07:30:00


In [38]:
morning_sightings['shape'].value_counts()

light        945
circle       604
disk         536
triangle     503
other        495
sphere       456
unknown      418
oval         386
fireball     356
cigar        283
changing     188
cylinder     165
formation    136
rectangle    133
diamond      107
flash         93
teardrop      89
egg           80
chevron       46
cone          27
cross         26
Name: shape, dtype: int64

Create a dataset that contains datetime, country, shape and duration_seconds. Save that dataset in csv format with filename simplified_dataset.csv.

In [39]:
# uncomment and complete the code below
# new_dataset = 
new_dataset = data.loc[:, ['datetime', 'country', 'shape', 'duration_seconds']]
new_dataset.head()

,datetime,country,shape,duration_seconds
0,1949-10-10 20:30:00,us,cylinder,2700.0
1,1949-10-10 21:00:00,NaN,light,7200.0
2,1955-10-10 17:00:00,gb,circle,20.0
3,1956-10-10 21:00:00,us,circle,20.0
4,1960-10-10 20:00:00,us,light,900.0


In [40]:
new_dataset.to_csv('simplified_dataset.csv', header=True, index=False)